*Author: [Daniel Puente Viejo](https://www.linkedin.com/in/danielpuenteviejo/)*

## **Create an Agent System with completly free LLM and Autogen for Course Recommendation**


In this tutorial, we will show how to use the LangGraph and Mistral to create a simple CV evaluator.


### <a id='1' style="color: skyblue;">**1 - Libraries**</a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import autogen
from autogen import UserProxyAgent, register_function, AssistantAgent

import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
def termination_message(msg):
    content = msg.get("content", "").lower()
    return "terminate" in content

# File path for the CV
cv_file_path = "data/cv.txt"

In [3]:
# It works FINE.
llama31_config = {
    "config_list": [
        {
            "model": "llama-3.1-8b-instant",
            "api_key": os.environ.get("GROQ_API_KEY"),
            "api_type": "groq"
        }
    ]
}

gemma2_config = {
    "config_list": [
        {
            "model": "gemma2-9b-it",
            "api_key": os.environ.get("GROQ_API_KEY"),
            "api_type": "groq"
        }
    ]
}

In [4]:
# The user proxy agent that executes tool calls.
user_proxy = UserProxyAgent(
    name="User",
    llm_config=False,
    is_termination_msg=termination_message,
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

In [5]:
## -- Agent 1: CV Analyser --
cv_analyser = AssistantAgent(
    name="CV Analyser",
    description="It analyzes the input it giving the most relevant information of the document.",
    system_message="""You must return an analysis of the input file.
    Other details for your answer:
    - Do not responde any piece of code.
    - Do not recommend courses just analyse the input file.""",
    llm_config=llama31_config,
)

## -- Agent 2: Skills Extractor --
skills_extractor = AssistantAgent(
    name="Skills Extractor",
    description="Returns a list of the candidates skills",
    system_message="""Your task is to extract the main skills from the input received.
    You must return the skills as a comma-separated list. For example: 'Python, Java, SQL'. Do not add more information, just the list of skills in the specified format.
    Other details for your answer:
    - Do not responde any piece of code.
    - Do not recommend courses just return the skills as a comma-separated list of the input file.""",
    llm_config=llama31_config,
)

## -- Agent 3: Courses Recommender --
courses_recommender = AssistantAgent(
    name="Courses recommender",
    description="The goal is to recommend courses. Read the courses from `data/courses.txt` and return the courses that match the skills of the input file.",
    system_message="""The goal is to recommend courses, for that follow these steps:
    1) Read the list of courses from `data/courses.txt` (use `read_txt_courses` tool).
    2) Combine the list of courses with the profile of the candidate.
    3) Return the courses that match the skills of the candidate.
    Other details for your answer: Do not responde any piece of code.""",
    llm_config=llama31_config,
)

# --- Agent 4: Resolution Checker Agent ---
resolution_checker = AssistantAgent(
    name="Resolution Checker Agent",
    description="Checks if the user's query has been resolved. If resolved, respond with 'TERMINATE', otherwise respond with 'CONTINUE'. Do not give more information just the word 'CONTINUE' or 'TERMINATE'.",
    system_message="""You should respond with 'TERMINATE' if the query has been resolved. Otherwise, respond with 'CONTINUE'. Do not add more information, just the word 'CONTINUE'.
    Try to be resolute, do not reply 'TERMINATE' if the initial query has not been resolved.
    **IMPORTANT**: Only respond with 'TERMINATE' if the query has been resolved, otherwise respond with 'CONTINUE'. If the answer is 'TERMINATE', you must answer 'TERMINATE' + generate a complete answer to the user's query considering all the information given by all the agents to generate the final answer.""",
    llm_config=gemma2_config,
    is_termination_msg=termination_message,

)

In [6]:
# Tool: Read CV from File
def read_txt(path: str) -> str:
    path = path.split("/")[-1]
    path = f"data/{path}"
    with open(path, "r") as file:
        return file.read()

In [7]:
# Register the read_txt function with the assistant agent.
register_function(
    read_txt,
    caller=cv_analyser,
    executor=user_proxy,
    name="read_txt_cv_analyse",
    description="Read the profile of the user",
)

register_function(
    read_txt,
    caller=courses_recommender,
    executor=user_proxy,
    name="read_txt_courses",
    description="Read available courses and/or the profile of the user",
)

register_function(
    read_txt,
    caller=skills_extractor,
    executor=user_proxy,
    name="read_txt_cv_skills",
    description="Read the profile of the user",
)

In [8]:
# Create a GroupChatManager to oversee the conversation
def custom_speaker_selection(last_speaker, groupchat):
    """
    Function to change the agent selection logic and customize the speaker selection.

    Args:
    last_speaker (Agent): The last agent that spoke.
    groupchat (GroupChat): The group chat object.

    Returns:
    Agent: The agent that will speak next.
    """
    agents_to_check = ["cv analyser", "skills extractor", "courses recommender"]

    # Extract the last and penultimate messages
    last_message = groupchat.messages[-1]
    penultimate_message = groupchat.messages[-2] if len(groupchat.messages) > 1 else None

    # Extract the names of the last and penultimate speakers
    last_speaker_name = last_speaker.name.lower()
    penultimate_speaker_name = penultimate_message['name'] if penultimate_message else None

    ### If the agent has made a tool call, call the 'User'
    if last_message.get("tool_calls", ""):
        return next(agent for agent in groupchat.agents if agent.name == "User")

    ### If the last speaker was the 'User' and the penultimate speaker was not the user or the Resolution Checker Agent, call the penultimate speaker
    ### Example: Skills Extractor makes use of a tool, so it call the 'User' agent. The 'User' agent then calls the 'Skills Extractor' to continue with the task it was doing.
    if last_speaker_name == "user" and penultimate_speaker_name and penultimate_speaker_name.lower() not in ["user", "resolution checker agent"]:
        return next(agent for agent in groupchat.agents if agent.name == penultimate_speaker_name)

    ### The an agent has given an answer, check if the quey is complete. For that call the 'Resolution Checker Agent'
    if last_speaker_name in agents_to_check:
        return next(agent for agent in groupchat.agents if agent.name == "Resolution Checker Agent")

    ### If no condition is met, leave the automatic configuration so that the next agent selection is based on an LLM.
    return 'auto'

In [20]:
groupchat = autogen.GroupChat(
    agents=[
        user_proxy,
        cv_analyser,
        skills_extractor,
        courses_recommender,
        resolution_checker,
    ],
    messages=[],
    allow_repeat_speaker=True,
    speaker_selection_method=custom_speaker_selection,
    role_for_select_speaker_messages="system"
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    code_execution_config={"use_docker": False},
    llm_config=gemma2_config,
    is_termination_msg=termination_message,
)

In [38]:
# Define the message to be sent to the manager
message = f"Recommend courses for the profile {cv_file_path}"
message = f"Analyse the following profile: {cv_file_path}."
message = f"Extract skills in comma-separated format based on the following profile: {cv_file_path}."
message = f"Recommends courses for the profile {cv_file_path}. Return also the profile skills in a list."

chat_result = user_proxy.initiate_chat(
    recipient=manager,
    message=message,
    max_turns=30,
)

User (to chat_manager):

Extract skills in comma-separated format based on the following profile: data/cv.txt.

--------------------------------------------------------------------------------
AUTO 4)

Next speaker: Skills Extractor

Skills Extractor (to chat_manager):

***** Suggested tool call (call_6v4h): read_txt_cv_skills *****
Arguments: 
{"path": "data/cv.txt"}
***************************************************************

--------------------------------------------------------------------------------
Tool 1)

Next speaker: User


>>>>>>>> EXECUTING FUNCTION read_txt_cv_skills...
User (to chat_manager):

User (to chat_manager):

***** Response from calling tool (call_6v4h) *****
Daniel Puente Viejo
+34 638 097 547
daniel.puenteviejo@gmail.com
https://github.com/DanielPuentee | https://www.linkedin.com/in/danielpuenteviejo | https://medium.com/@daniel.puenteviejo | https://huggingface.co/D0k-tor

---

PROFESSIONAL EXPERIENCE

NTT Data
➢ Clients: Cepsa, Repsol, Unsere Grüne Gla

In [40]:
agent_order = [x['name'] for x in chat_result.chat_history]
print(agent_order)

['User', 'Skills Extractor', 'User', 'Skills Extractor', 'Resolution Checker Agent', 'Courses recommender', 'Resolution Checker Agent', 'User', 'CV Analyser', 'Resolution Checker Agent']


In [41]:
print(chat_result.chat_history[-1]['content'])

TERMINATE  

Skills: Python, R, SQL, Langchain, CrewAI, Ollama, Hugging Face, Pytorch, Azure, AWS, Azure AI Services, Bedrock, Finetuning, RAG, CUDA, Git, Databricks, GNN, QLoRA   


---